In [1]:
from itertools import chain
from src_draft.utils import LOW_IMP_FEATURES, ZERO_IMP_FEATURES
import shared.ml_config_core as ml_config_core
import pandas as pd
from shared.ml_config_core import ModelConfigsCollection
from shared.ml_config_runner import run_tuning_for_configs_collection
from shared.definitions import TuningResult

from Draft import feature_builder_v2

import importlib

VERBOSE = True
importlib.reload(feature_builder_v2)
features_matrix = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                        ds_type=feature_builder_v2.DatasetType.BASE)
display("Distribution of Samples")
value_counts = features_matrix["TARGET"].value_counts()
df_value_counts = pd.DataFrame(
    {"Total Rows": value_counts, "Proportion": value_counts / len(features_matrix)}
)
df_value_counts.round(2)
transformers = [
    # No feature engineering tuning for for rejected/accepted df
    # ml_config_core.AcceptedRejectedDropAllButEmployment,  # Only used for testing
]

baseline_only_credit_ratings_config = {
    # Only uses  ExtScore1/2/3
    "Baseline_Only_CreditRatings": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.BASE_ONLY_CREDIT_RATINGS
        })
}

baseline_only_base_feats = {
    # Only includes base table (i.e. no aggregations from client history)
    "LGBM_AUC_Base_Features": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.BASE,
        }),
}

weighted_log_loss_config = {
    # Tuned on score combined from logloss and F1 score
    "LGBM_Weighted_LogLoss": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneWeightedLogLossF1(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

tune_auc_config: ModelConfigsCollection = {
    "LGBM_AUC": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.FULL,
            "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

dart_tune_auc_config: ModelConfigsCollection = {
    "LGBM_Dart_AUC": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMDartTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.FULL,
            "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

tune_auc_all_features_config: ModelConfigsCollection = {
    "LGBM_AUC_All_Features": ml_config_core.ModelPipelineConfig(
        model_config=ml_config_core.LGBMTuneAUC(),
        transformer_config=ml_config_core.PipelineTransformerConfig(transformers),
        data_loader_params={
            "drop_meta_data": True,
            "ds_type": feature_builder_v2.DatasetType.FULL,
            # "drop_cols_post_proc": ZERO_IMP_FEATURES + LOW_IMP_FEATURES
        }),
}

configs: ModelConfigsCollection = {
    **baseline_only_credit_ratings_config,
    # **baseline_only_base_feats,
    # **weighted_log_loss_config,
    # **tune_auc_config,
    # **dart_tune_auc_config,
    # **tune_auc_all_features_config
}

Full DS size: 307511


'Distribution of Samples'

In [2]:
configs.keys()

dict_keys(['Baseline_Only_CreditRatings'])

In [3]:
tuning_results = run_tuning_for_configs_collection(configs, feature_builder_v2.load_datasets_and_prepare_features)

[I 2024-04-29 21:58:59,247] A new study created in memory with name: no-name-b2577545-35dc-460c-858e-a1418afeeec7


Full DS size: 307511
Tunning: - transformers: transformers: 0
 total options: 0
[]
search_grid:
{}

 - model: Baseline_Only_CreditRatings n_iters=10 with:
 {'model__n_estimators': Range(50, 1000, 50, int), 'model__learning_rate': Range(0.01, 0.3, 0.01, float), 'model__max_depth': Range(3, 11, 1, int), 'model__num_leaves': Range(8, 256, 8, int), 'model__min_gain_to_split': Range(0.0, 15.0, 0.5, float), 'model__min_data_in_leaf': Range(0, 3000, 100, int), 'model__lambda_l1': Range(0, 110, 5, int), 'model__lambda_l2': Range(0, 110, 5, int), 'model__bagging_fraction': Range(0.2, 1.0, 0.1, float), 'model__feature_fraction': Range(0.2, 1.0, 0.1, float), 'model__max_bin': Range(50, 500, 25, int)}
Fold: Tuning: n_train=263580, eval_set=43931
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, e

[I 2024-04-29 21:59:01,822] Trial 0 finished with value: 0.723379541466732 and parameters: {'model__n_estimators': 850, 'model__learning_rate': 0.12, 'model__max_depth': 9, 'model__num_leaves': 256, 'model__min_gain_to_split': 8.5, 'model__min_data_in_leaf': 1100, 'model__lambda_l1': 10, 'model__lambda_l2': 105, 'model__bagging_fraction': 0.7, 'model__feature_fraction': 0.30000000000000004, 'model__max_bin': 400}. Best is trial 0 with value: 0.723379541466732.


Fold: Tuning: n_train=263580, eval_set=43931
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Tune: val_score:0.7200, std_test_score:0.00654 train_set_score:0.7236
folds val/train: [0.7221, 0.7225, 0.7219, 0.7154, 0.7282, 0.7237, 0.7064] / [0.7249, 0.7249, 0.7249, 0.7258, 0.7239, 0.7247, 0.7162], mean fold time: 0.57


[I 2024-04-29 21:59:06,546] Trial 1 finished with value: 0.7200250827430056 and parameters: {'model__n_estimators': 150, 'model__learning_rate': 0.02, 'model__max_depth': 11, 'model__num_leaves': 152, 'model__min_gain_to_split': 6.0, 'model__min_data_in_leaf': 900, 'model__lambda_l1': 95, 'model__lambda_l2': 70, 'model__bagging_fraction': 0.30000000000000004, 'model__feature_fraction': 0.8, 'model__max_bin': 225}. Best is trial 0 with value: 0.723379541466732.


...

Fold: Tuning: n_train=263580, eval_set=43931
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Tune: val_score:0.7239, std_test_score:0.00366 train_set_score:0.7254
folds val/train: [0.7241, 0.7247, 0.7236, 0.7189, 0.7307, 0.726, 0.7198] / [0.7255, 0.7254, 0.7255, 0.7261, 0.7242, 0.7253, 0.7259], mean fold time: 0.21


[I 2024-04-29 22:08:49,734] Trial 248 finished with value: 0.7239381035171804 and parameters: {'model__n_estimators': 350, 'model__learning_rate': 0.25, 'model__max_depth': 3, 'model__num_leaves': 232, 'model__min_gain_to_split': 2.0, 'model__min_data_in_leaf': 0, 'model__lambda_l1': 0, 'model__lambda_l2': 55, 'model__bagging_fraction': 0.30000000000000004, 'model__feature_fraction': 0.2, 'model__max_bin': 125}. Best is trial 230 with value: 0.7239502969915346.


Fold: Tuning: n_train=263580, eval_set=43931
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Fold: Tuning: n_train=263581, eval_set=43930
Tune: val_score:0.7237, std_test_score:0.00343 train_set_score:0.7254
folds val/train: [0.724, 0.7246, 0.7233, 0.719, 0.7299, 0.7256, 0.7195] / [0.7254, 0.7254, 0.7252, 0.7264, 0.7239, 0.7253, 0.7261], mean fold time: 0.22


[I 2024-04-29 22:08:52,046] Trial 249 finished with value: 0.7236990920618321 and parameters: {'model__n_estimators': 400, 'model__learning_rate': 0.25, 'model__max_depth': 3, 'model__num_leaves': 240, 'model__min_gain_to_split': 2.0, 'model__min_data_in_leaf': 0, 'model__lambda_l1': 0, 'model__lambda_l2': 55, 'model__bagging_fraction': 0.30000000000000004, 'model__feature_fraction': 0.2, 'model__max_bin': 150}. Best is trial 230 with value: 0.7239502969915346.



 -- -- --
 TUNING RESULT: 
 Baseline_Only_CreditRatings: auc = 0.7239502969915346 params: {'model__n_estimators': 300, 'model__learning_rate': 0.25, 'model__max_depth': 3, 'model__num_leaves': 224, 'model__min_gain_to_split': 1.5, 'model__min_data_in_leaf': 100, 'model__lambda_l1': 0, 'model__lambda_l2': 50, 'model__bagging_fraction': 1.0, 'model__feature_fraction': 0.30000000000000004, 'model__max_bin': 125}
 -- --

Baseline_Only_CreditRatings Fit, total time:592.8



In [4]:
for t, v in tuning_results.items():
    display(t)
    display(v.hyper_param_all_cv_results)



'LGBM_AUC'

,Trial Number,Mean Test Score (AUC),mean_test_score,mean_train_score,std_test_score,macro_f1,micro_f1,f1_target1,precision_target1,recall_target1,log_loss,pr_auc,mean_fold_time,Parameters
231,231,0.776092,0.776092,0.843902,0.003024,0.566457,0.740422,0.291828,0.187127,0.662518,0.527647,0.263602,6.471375,"{'model__objective': 'binary', 'model__class_w..."
193,193,0.775960,0.775960,0.849419,0.003408,0.567945,0.742949,0.292970,0.188295,0.659698,0.524256,0.263453,6.741207,"{'model__objective': 'binary', 'model__class_w..."
161,161,0.775731,0.775731,0.833488,0.003669,0.563487,0.735092,0.289794,0.184919,0.669486,0.534421,0.263478,5.783973,"{'model__objective': 'binary', 'model__class_w..."
227,227,0.775717,0.775717,0.842292,0.003251,0.565909,0.739749,0.291205,0.186638,0.662236,0.528503,0.263391,5.505174,"{'model__objective': 'binary', 'model__class_w..."
190,190,0.775712,0.775712,0.848627,0.003517,0.567445,0.742686,0.292125,0.187762,0.657684,0.524701,0.263422,6.676134,"{'model__objective': 'binary', 'model__class_w..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44,44,0.759950,0.759950,0.775641,0.012737,0.541356,0.701113,0.270669,0.168534,0.687009,0.595431,0.226876,5.571353,"{'model__objective': 'binary', 'model__class_w..."
128,128,0.756286,0.756286,0.781508,0.003210,0.542204,0.704303,0.269791,0.168484,0.676657,0.599644,0.235728,1.633354,"{'model__objective': 'binary', 'model__class_w..."
174,174,0.748395,0.748395,0.768160,0.003632,0.536500,0.697191,0.263589,0.163990,0.671299,0.661424,0.216286,1.595065,"{'model__objective': 'binary', 'model__class_w..."
146,146,0.747943,0.747943,0.765273,0.003534,0.535045,0.694596,0.262677,0.163133,0.673877,0.664139,0.215530,1.512120,"{'model__objective': 'binary', 'model__class_w..."


'LGBM_Dart_AUC'

,Trial Number,Mean Test Score (AUC),mean_test_score,mean_train_score,std_test_score,macro_f1,micro_f1,f1_target1,precision_target1,recall_target1,log_loss,pr_auc,mean_fold_time,Parameters
242,242,0.777050,0.777050,0.823991,0.003522,0.561941,0.731262,0.289595,0.184081,0.678510,0.539715,0.265901,18.134199,"{'model__boosting_type': 'dart', 'model__objec..."
219,219,0.776988,0.776988,0.827738,0.003543,0.562456,0.732562,0.289638,0.184349,0.675368,0.537131,0.265637,22.858985,"{'model__boosting_type': 'dart', 'model__objec..."
181,181,0.776971,0.776971,0.820890,0.003434,0.560811,0.729519,0.288608,0.183202,0.679637,0.541838,0.265183,18.932894,"{'model__boosting_type': 'dart', 'model__objec..."
148,148,0.776938,0.776938,0.825691,0.003266,0.562256,0.732013,0.289657,0.184257,0.676818,0.538614,0.265921,18.012119,"{'model__boosting_type': 'dart', 'model__objec..."
223,223,0.776937,0.776937,0.827678,0.003326,0.562959,0.732803,0.290510,0.184888,0.677623,0.537636,0.266211,20.997200,"{'model__boosting_type': 'dart', 'model__objec..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,2,0.762608,0.762608,0.782228,0.003897,0.545220,0.707149,0.273850,0.171193,0.684028,0.578606,0.243208,4.888875,"{'model__boosting_type': 'dart', 'model__objec..."
5,5,0.760132,0.760132,0.783908,0.003648,0.543841,0.705676,0.272139,0.170011,0.681571,0.588143,0.241381,8.437506,"{'model__boosting_type': 'dart', 'model__objec..."
164,164,0.758951,0.758951,0.772656,0.003895,0.540465,0.700160,0.269567,0.167779,0.685358,0.580540,0.241095,1.428431,"{'model__boosting_type': 'dart', 'model__objec..."
56,56,0.756898,0.756898,0.773740,0.003494,0.539982,0.700499,0.268246,0.167077,0.680000,0.585412,0.238343,21.932889,"{'model__boosting_type': 'dart', 'model__objec..."


In [5]:
tuning_results.keys()

dict_keys(['LGBM_Dart_AUC', 'LGBM_AUC'])

In [6]:
# Results:

# Only main dataset:
# - top res      : 0.753
# - full + gbdt  : 0.765
# - full + dart  : 0.766


In [7]:
from Draft.feature_builder_v2 import DatasetType

importlib.reload(feature_builder_v2)
features_matrix_full = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                             ds_type=DatasetType.FULL)


C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
 

Appending previous history
Full DS size: 307511


In [8]:
features_matrix_full

,TARGET,NameContractType,CodeGender,FlagOwnCar,FlagOwnRealty,CntChildren,AmtIncomeTotal,AmtCredit,AmtAnnuity,AmtGoodsPrice,...,PrevCodeRejectReasonHcCount,PrevCodeRejectReasonLimitCount,PrevCodeRejectReasonScoCount,PrevCodeRejectReasonScofrCount,PrevCodeRejectReasonSystemCount,PrevCodeRejectReasonVerifCount,PrevCodeRejectReasonXapCount,PrevCodeRejectReasonXnaCount,PrevLastLoanNflagInsuredOnApproval,PrevAvgNflagInsuredOnApproval
0,1,Cash loans,M,False,True,0,202500.0,406597.5,24700.5,351000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
1,0,Cash loans,F,False,False,0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.666667
2,0,Revolving loans,M,True,True,0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
3,0,Cash loans,F,False,True,0,135000.0,312682.5,29686.5,297000.0,...,0.0,1.0,0.0,0.0,0.0,0.0,8.0,0.0,NaN,0.000000
4,0,Cash loans,M,False,True,0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,Cash loans,M,False,False,0,157500.0,254700.0,27558.0,225000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
307507,0,Cash loans,F,False,True,0,72000.0,269550.0,12001.5,225000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.000000
307508,0,Cash loans,F,False,True,0,153000.0,677664.0,29979.0,585000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.500000
307509,1,Cash loans,F,False,True,0,171000.0,370107.0,20205.0,319500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.500000


In [9]:
from Draft.feature_builder_v2 import DatasetType

importlib.reload(feature_builder_v2)
features_matrix_full = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                             ds_type=DatasetType.FULL)
features_matrix_base = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                             ds_type=DatasetType.BASE)


C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(
C:\Users\Paulius\AppData\Local\pypoetry\Cache\virtualenvs\ppuodz-ml-4-1-dqELbViF-py3.12\Lib\site-packages\woodwork\type_sys\utils.py:40: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
 

Appending previous history
Full DS size: 307511
Full DS size: 307511


In [10]:
features_matrix_full

,TARGET,NameContractType,CodeGender,FlagOwnCar,FlagOwnRealty,CntChildren,AmtIncomeTotal,AmtCredit,AmtAnnuity,AmtGoodsPrice,...,PrevCodeRejectReasonHcCount,PrevCodeRejectReasonLimitCount,PrevCodeRejectReasonScoCount,PrevCodeRejectReasonScofrCount,PrevCodeRejectReasonSystemCount,PrevCodeRejectReasonVerifCount,PrevCodeRejectReasonXapCount,PrevCodeRejectReasonXnaCount,PrevLastLoanNflagInsuredOnApproval,PrevAvgNflagInsuredOnApproval
0,1,Cash loans,M,False,True,0,202500.0,406597.5,24700.5,351000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
1,0,Cash loans,F,False,False,0,270000.0,1293502.5,35698.5,1129500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.666667
2,0,Revolving loans,M,True,True,0,67500.0,135000.0,6750.0,135000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
3,0,Cash loans,F,False,True,0,135000.0,312682.5,29686.5,297000.0,...,0.0,1.0,0.0,0.0,0.0,0.0,8.0,0.0,NaN,0.000000
4,0,Cash loans,M,False,True,0,121500.0,513000.0,21865.5,513000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,1.0,0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0,Cash loans,M,False,False,0,157500.0,254700.0,27558.0,225000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.000000
307507,0,Cash loans,F,False,True,0,72000.0,269550.0,12001.5,225000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.000000
307508,0,Cash loans,F,False,True,0,153000.0,677664.0,29979.0,585000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.500000
307509,1,Cash loans,F,False,True,0,171000.0,370107.0,20205.0,319500.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,1.0,0.500000


In [11]:
importlib.reload(feature_builder_v2)
features_matrix = feature_builder_v2.load_datasets_and_prepare_features(drop_meta_data=True,
                                                                        ds_type=feature_builder_v2.DatasetType.BASE_ONLY_CREDIT_RATINGS)
from shared import stats_utils

stats_utils.nan_summary(features_matrix)

Full DS size: 307511


,Total NaN Values,Proportion NaN (%)
ExtSource1,173378,56.0
ExtSource2,660,0.0
ExtSource3,60965,20.0
